# Homework Week 2

## Q1. Install MLflow

In [1]:
import mlflow


In [2]:
t = !mlflow --version

In [3]:
print(f"The version of MLflow is {t.fields(2)}")

The version of MLflow is ['1.26.1']


## Q2. Download and preprocess the data

In [4]:
!python preprocess_data.py --raw_data_path ~/notebooks/mlops-zoomcamp/week2/data/ --dest_path ./output

In [5]:
!ls ./output

dv.pkl	test.pkl  train.pkl  valid.pkl


In [6]:
print("In total there are 4 files created in the output.")

In total there are 4 files created in the output.


# Q3. Train a model with autolog

In [8]:
!python train.py

2022/05/31 09:00:45 INFO mlflow.tracking.fluent: Experiment with name 'nyc-experiment-homework-w2' does not exist. Creating a new experiment.
2022/05/31 09:01:11 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/home/ubuntu/anaconda3/envs/exp-tracking-env/lib/python3.9/site-packages/_distutils_hack/__init__.py:30: UserWarning: Setuptools is replacing distutils."


In [9]:
# connect to client to retrieve number of parameter logged
from mlflow.tracking import MlflowClient
TRACKING_URI = "sqlite:///mlflow.db"
client = MlflowClient(tracking_uri = TRACKING_URI )

client.list_experiments()

[<Experiment: artifact_location='./mlruns/0', experiment_id='0', lifecycle_stage='active', name='Default', tags={}>,
 <Experiment: artifact_location='./mlruns/2', experiment_id='2', lifecycle_stage='active', name='nyc-experiment-homework-w2', tags={}>]

In [15]:
client.list_run_infos(experiment_id=2)

[<RunInfo: artifact_uri='./mlruns/2/4537467a7cf448a4af17ab0ac478d4e7/artifacts', end_time=1653987671791, experiment_id='2', lifecycle_stage='active', run_id='4537467a7cf448a4af17ab0ac478d4e7', run_uuid='4537467a7cf448a4af17ab0ac478d4e7', start_time=1653987646247, status='FINISHED', user_id='ubuntu'>]

In [20]:
run = client.get_run(run_id ='4537467a7cf448a4af17ab0ac478d4e7' )

params = run.data.params
params

{'bootstrap': 'True',
 'ccp_alpha': '0.0',
 'criterion': 'squared_error',
 'max_depth': '10',
 'max_features': '1.0',
 'max_leaf_nodes': 'None',
 'max_samples': 'None',
 'min_impurity_decrease': '0.0',
 'min_samples_leaf': '1',
 'min_samples_split': '2',
 'min_weight_fraction_leaf': '0.0',
 'n_estimators': '100',
 'n_jobs': 'None',
 'oob_score': 'False',
 'random_state': '0',
 'verbose': '0',
 'warm_start': 'False'}

In [23]:
print(f"The number of parameters automatically logged is {len(params)}.")

The number of parameters automatically logged is 17.


# Q4. Launch the tracking server locally

In [27]:
print("In addition to backend-store-uri we need also to pass the artifacts-destination in order to configure properly the server.")

In addition to backend-store-uri we need also to pass the artifacts-destination in order to configure properly the server.


# Q5. Tune the hyperparameters of the model

In [97]:
!python hpo.py

100%|█████████| 50/50 [08:51<00:00, 10.63s/trial, best loss: 6.6284257482044735]


In [98]:
from mlflow.entities import ViewType

runs = client.search_runs(
    experiment_ids= 3,
    filter_string="",
    run_view_type= ViewType.ACTIVE_ONLY,
    max_results= 6,
    order_by= ["metrics.rmse ASC"]
)

for run in runs:
    print(f"run id : {run.info.run_id}, rmse : {run.data.metrics['rmse']:.3f}")


run id : 56c4c8a390074e7fb6b768bd653219ec, rmse : 6.628
run id : 6baa81164485481782b1cbeb9ef9c71b, rmse : 6.630
run id : cb3f9a8e4e1c47c280e075afdbbeb873, rmse : 6.630
run id : 4bfc518227ab4a60b6872e3b6780ddfb, rmse : 6.630
run id : af69d31010284108bbc2db95ad5a9ac4, rmse : 6.630
run id : 5f46c149d1854bc1ad42d3225e523b6d, rmse : 6.631


In [99]:
print(f"The best validation RMSE is {runs[0].data.metrics['rmse']:.3f}.")

The best validation RMSE is 6.628.


# Q6. Promote the best model to the model registry

In [112]:
!python register_model.py 

Error processing line 1 of /home/ubuntu/anaconda3/envs/exp-tracking-env/lib/python3.9/site-packages/distutils-precedence.pth:

  Traceback (most recent call last):
    File "/home/ubuntu/anaconda3/envs/exp-tracking-env/lib/python3.9/site.py", line 169, in addpackage
      exec(line)
    File "<string>", line 1, in <module>
  ModuleNotFoundError: No module named '_distutils_hack'

Remainder of file ignored
Registered model 'best-model-week-2' already exists. Creating a new version of this model...
2022/05/31 13:37:05 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: best-model-week-2, version 1
Created version '1' of model 'best-model-week-2'.


In [113]:
from mlflow.entities import ViewType

runs = client.search_runs(
    experiment_ids= 4,
    filter_string="",
    run_view_type= ViewType.ACTIVE_ONLY,
    max_results= 6,
    order_by= ["metrics.test_rmse ASC"]
)

for run in runs:
    print(f"run id : {run.info.run_id}, rmse : {run.data.metrics['test_rmse']:.3f}")


run id : bcdd3f2a819948e5b2710b9e5985792e, rmse : 6.548
run id : 47a465091e124ab1b1e84aad86179a0e, rmse : 6.549
run id : f640c29f3b7f4095b2c9a13890b4fdd7, rmse : 6.549
run id : 8f4d7a79852f492dbb9dd12f7aa26df1, rmse : 6.549
run id : dd4798c0e7624f1aaa836d6062e1956e, rmse : 6.550


In [115]:
print(f"The best test RMSE is {runs[0].data.metrics['test_rmse']:.2f}.")

The best test RMSE is 6.55.
